In [1]:
import numpy as np
import torch
import torchvision
from torch import nn
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import ResNet152_Weights, resnet152
from tqdm import tqdm

In [2]:
RANDOM_STATE = 2025
BATCH_SIZE = 16
LEARNING_RATE = 1e-5
# LEARNING_RATE = 5e-6
NO_OF_ITERATIONS = 300
CUDA_DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
PATH_TO_MODEL_SAVE = "resnet_model_weight_for_Flowers_Without_Weight_Freeze.pth"

In [3]:
transforms_functions = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.RandomCrop((224, 224)),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=30),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [4]:
transforms_functions_for_test = transforms.Compose(
    [
        transforms.Resize((244, 244)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [5]:
dataset_of_cnc = ImageFolder(
    root="FlowerImageDatasetArchive/train", transform=transforms_functions
)

In [6]:
dataset_of_cnc.classes

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [7]:
generator_for_random_split = torch.Generator().manual_seed(RANDOM_STATE)

In [8]:
training_dataset, test_dataset = random_split(
    dataset_of_cnc, [0.7, 0.3], generator=generator_for_random_split
)

In [9]:
dataloader_for_flower_training = DataLoader(
    training_dataset, batch_size=BATCH_SIZE, shuffle=True
)

In [10]:
dataloader_for_flower_test = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False
)

In [11]:
resnet_model_for_cnc = resnet152(weights=ResNet152_Weights.DEFAULT)
# resnet_model_for_cnc = resnet152()

In [12]:
resnet_model_for_cnc

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
# resnet_model_for_cnc.requires_grad_(requires_grad = False)

In [14]:
resnet_model_for_cnc.fc.requires_grad_(requires_grad=True)

Linear(in_features=2048, out_features=1000, bias=True)

In [15]:
resnet_model_for_cnc.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=2048, out_features=512),
    torch.nn.ReLU(),
    torch.nn.Dropout(p=0.2),
    torch.nn.Linear(
        in_features=512,
        out_features=len(dataset_of_cnc.classes),
    ),
    torch.nn.Softmax(dim=1),
)

In [16]:
for module in resnet_model_for_cnc.modules():
    if type(module) == nn.Linear:
        nn.init.kaiming_normal_(
            module.weight.data,
            a=0,
            mode="fan_out",
            nonlinearity="relu",
        )
        if module.bias is not None:
            fan_in, fan_out = nn.init._calculate_fan_in_and_fan_out(module.weight.data)
            bound = 1 / (fan_out) ** 0.5
            nn.init.normal_(module.bias, -bound, bound)

In [17]:
resnet_model_for_cnc.to(device=CUDA_DEVICE)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [18]:
# resnet_model_for_cnc.load_state_dict(torch.load(PATH_TO_MODEL_SAVE, map_location = CUDA_DEVICE))

In [19]:
torch.cuda.is_available()

True

In [20]:
optimizer = Adam(resnet_model_for_cnc.parameters(), lr=LEARNING_RATE)

In [21]:
loss_function = CrossEntropyLoss()

In [22]:
for i_iteration in tqdm(range(NO_OF_ITERATIONS)):
    resnet_model_for_cnc.train()
    training_loss = []
    test_loss = []
    for i_image, i_target in dataloader_for_flower_training:
        i_image = i_image.to(device=CUDA_DEVICE)
        i_target = i_target.to(device=CUDA_DEVICE)
        optimizer.zero_grad()

        output = resnet_model_for_cnc(i_image)
        loss = loss_function(output, i_target)
        loss.backward()
        optimizer.step()

        training_loss.append(loss.item())

    print("Training loss at", i_iteration, "iteration is", np.mean(training_loss))

    torch.save(resnet_model_for_cnc.state_dict(), PATH_TO_MODEL_SAVE)
    resnet_model_for_cnc.eval()
    for test_image, test_target in dataloader_for_flower_test:
        test_image = test_image.to(device=CUDA_DEVICE)
        test_target = test_target.to(device=CUDA_DEVICE)
        test_prediction = resnet_model_for_cnc(test_image)

        temp_test_loss = loss_function(test_prediction, test_target)

        test_loss = temp_test_loss.item()

    print("***", "Test loss at", i_iteration, "iteration is", np.mean(test_loss))

  0%|                                                   | 0/300 [00:00<?, ?it/s]

Training loss at 0 iteration is 1.5794785712376114


  0%|▏                                        | 1/300 [00:29<2:25:26, 29.18s/it]

*** Test loss at 0 iteration is 1.3383524417877197
Training loss at 1 iteration is 1.4275552123046118


  1%|▎                                        | 2/300 [00:58<2:24:41, 29.13s/it]

...

*** Test loss at 297 iteration is 1.021481990814209
Training loss at 298 iteration is 0.9054806744756777


100%|██████████████████████████████████████▊| 299/300 [2:27:52<00:28, 28.62s/it]

*** Test loss at 298 iteration is 1.0477343797683716
Training loss at 299 iteration is 0.9054721771192945


100%|███████████████████████████████████████| 300/300 [2:28:21<00:00, 29.67s/it]

*** Test loss at 299 iteration is 1.0546525716781616
